# Rapport TAL
## Partie Présidents
Membres du binôme :
- KRISNI Almehdi (3800519)
- ARICHANDRA Santhos (3802651)

### Introduction
Dans cette partie, nous allons nous intéresser à la prédiction de l'interlocuteur lors d'un débat présidentiel entre Chirac et Mitterrand. Entre d'autres termes, qui a dit quoi ?

Nous disposons donc d'une base de données d'apprentissage de 57000 lignes représentant un ensemble de phrases, chacune de ces dernières étiquetées par un C (Chirac) ou un M (Mitterrand), désignant la personne l'ayant prononcé. On peut alors labéliser notre ensemble d'apprentissage en 2 classes distinctes. 

Notre objectif est de mettre en place des algorithmes allant nous permettre de labelliser de nouveaux débats entre Chirac et Mitterrand.

### Mise en place des import et des librairies utilisées

In [59]:
import numpy as np
import matplotlib.pyplot as plt

import codecs
import re
import os.path

from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop

from collections import Counter

import re
import unicodedata
import string
from nltk.corpus import stopwords

from collections import Counter

from sklearn.feature_extraction import text 

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import numpy as np
import sklearn.naive_bayes as nb
from sklearn import svm
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV

# Importation des librairies standards:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  

# Importation de votre librairie iads
import sys
sys.path.append('../')

# Iportation de utils
from iads import utils as ut

# commande TRES utile pour recharger automatiquement le code que vous modifiez dans les modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Première analyse des données

On extrait dans un premier les informations à la base de données d'apprentissage. On utilise la fonction d'extraction "load_pres" située dans le fichier utils.py.

On considère 2 classes, la classe -1 représentant Mitterrand et la classe +1 représentant Chirac.

In [65]:
appdata = ut.load_pres("AFDpresidentutf8/corpus.tache1.learn.utf8")
appX = appdata[0]
appY = appdata[1]

On s'intéresse à la répartition des labels dans les données. Une personne parle-t-elle beaucoup plus que l'autre ?

In [66]:
print("Le nombre de lignes prononcées par Mitterrand :", str(appY.count(-1)))
print("Le nombre de lignes prononcées par Chirac :", str(appY.count(1)))
print("Proportion de Chirac dans la base d'apprentissage :", str((appY.count(1) / len(appY)) * 100), "%")
print("Proportion de Mitterrand dans la base d'apprentissage :", str((appY.count(-1) / len(appY)) * 100), "%")

Le nombre de lignes prononcées par Mitterrand : 7523
Le nombre de lignes prononcées par Chirac : 49890
Proportion de Chirac dans la base d'apprentissage : 86.89669587027329 %
Proportion de Mitterrand dans la base d'apprentissage : 13.103304129726718 %


On remarque donc qu'il y a presque 87% des lignes dans la base de données d'apprentissage ayant été prononcées par Mitterrand. Il se peut que cette différence puisse jouer un rôle important lors de l'apprentissage.

### Nettoyage des données

On effectue alors un premier nettoyage des données en supprimant les chiffres, la ponctuation, les accents, les majuscules et les stopwords de la langue française.

In [62]:
appX = appdata[0]
appY = appdata[1]
# appX = ut.transform(appX, punc=True, accentMaj=True, nb=True, stopW=True)
print(appX[:5])

[" Quand je dis chers amis, il ne s'agit pas là d'une formule diplomatique, mais de l'expression de ce que je ressens.\n", " D'abord merci de cet exceptionnel accueil que les Congolais, les Brazavillois, nous ont réservé cet après-midi.\n", " C'est toujours très émouvant de venir en Afrique car c'est probablement l'une des rares terres du monde où l'on ait conservé cette convivialité, cette amitié, ce respect de l'autre qui s'expriment avec chaleur, avec spontanéité et qui réchauffent le coeur de ceux qui arrivent et de ceux qui reçoivent.\n", ' Aucun citoyen français ne peut être indifférent à un séjour à Brazzaville.\n', ' Le Congo, que naguère le <nom> qualifia de "refuge pour la liberté", de "base de départ pour la libération", de "môle pour la Résistance", comment ne pas être heureux de s\'y retrouver ?\n']


On crée un Counter allant mettre en avant les mots les plus souvent utilisés par chacun des Présidents, après nettoyage des données avec les paramètres selectionnés.

In [85]:
indexC = list(np.where(np.array(appY) == (1)))
indexM = list(np.where(np.array(appY) == (-1)))
print(np.array([appY[i] for i in indexC]).flatten())
# counterC = Counter(np.array([appY[i] for i in indexC]).flatten())

NameError: name 'where' is not defined

On s'intéresse à mettre en place deux dictionnaires exclusifs pour Chirac et Mitterrand. Il s'agit de dictionnaires comportant les mots qu'ils utilisent de manière exclusive.